<a href="https://colab.research.google.com/github/LambdaTheda/DS-Unit-2-Linear-Models/blob/master/S_mar21_1AM_REGRSS2_unit2_spr1_mod2_212_assmnt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
pd.set_option('display.max_rows', None)

df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
df.head(200)

In [0]:
df

In [0]:
df.shape

In [0]:
# Train/test split. Use data from April & May 2016 to train. 

df['created'] = pd.to_datetime(df['created'])
train = df[(df['created'].dt.year == 2016) & (df['created'].dt.month.isin([4, 5]))]
train.shape


In [0]:
# explore column to use for train/test split
df['created'].value_counts(48817) # returns counts as < 1 (bc of dt classes? or https://www.w3resource.com/pandas/series/series-value_counts.php- SERIES notdf fcngit)


In [0]:
df['created'].value_counts(48817).head(300)

In [0]:
# explore column to use for train/test split
df['created'].describe()

In [0]:
df['created'].shape

#  train/test split

In [0]:
#ATTEMPT 3: Parsing out only YEAR from df['created']-
df['created'] = pd.to_datetime(df['created']) #dot notation: df['created'] = pd.to_datetime(df.created); from https://www.youtube.com/watch?v=yCgJGsg0Xa4
df['year'] = df['created'].dt.year
df['year']

In [0]:
#temp df for making train set from df with df['created'].dt.year == 2016
df_train_yr = df[(df['year'] == 2016)]
df_train_yr.head(200)


In [0]:
df_train_yr.nunique

In [0]:
#df_train_yr.value_counts() #AttributeError: 'DataFrame' object has no attribute 'value_counts'; BC NO CLASSES ?#


In [0]:
#filter df_train_yr for April and May rows- WORKS
df_train_yr_and_months = df_train_yr[df_train_yr['created'].dt.month.isin([4, 5])]  #df[(df['year'] == 2016)]
df_train_yr_and_months.head(200)          

In [0]:
df_train_yr_and_months.sample(300)

In [0]:
df_train = df_train_yr_and_months
df_train.head(300)

In [0]:
df_train.shape

# FILTER df_train TO MAKE TEST SET USING JUNE 2016 ROWS

In [0]:
df['created'] = pd.to_datetime(df['created'])
df['created'].dtype

In [0]:
# ATTEMPT 2: AttributeError: 'str' object has no attribute 'dt'
'''
df['created'] = pd.to_datetime(df['created'])

df['month'] = df['created'].dt.month

df_test = df_train_yr[df_train_yr['created'.dt.month] == 6] 
df_test
'''
 

In [0]:
# ATTEMPT 3: WORKS

df['created'] = pd.to_datetime(df['created'])
#df['created'].dtype            # RETURNS dtype('<M8[ns]')
df_test = df_train_yr[df_train_yr['created'].dt.month.isin([6])]
df_test.head(300)

In [0]:
df_test.shape

In [0]:
df_test['created'].value_counts()

# "my own" S t     r            e           T                c  h                                          GOAL!  3d plot of 3 features


In [0]:
import plotly.express as px
px.scatter_3d(
    df,
    x = 'bedrooms',
    y = 'price',
    z = 'bathrooms', 
    title = 'NYC Apartment Rental Prices, 2016'
)

# Engineer at least two new features. 

In [0]:
# 1) Does the apartment have a description? ATTEMPTS/Exploration

# Using for loop
# for descrp in range (len(df['description'])):  

# Chris suggests apply():

 # use regex to find '     ' string that seems to represent no descriptions
'''
import re 
no = re.compile('        ')
no    # RETURNS: re.compile(r'        ', re.UNICODE)
'''

# 1) FEATURE ENGINEERED 'has_description' column:

In [0]:
# 1) FEATURE ENGINEERED 'has_description' column: Does the apartment have a description?
# set the new column to the dataframe column with all values EXCEPT '        '
df['has_description'] = ~df['description'].isin(['        '])

df['has_description'] = df['has_description'].map({True: 'Yes', False: 'No'})

df['has_description']

In [0]:
df.sample(200)

In [0]:
# use findall() 
'''
no.findall('        ')
print(df['description'].no.findall('        ')) # RETURNS: AttributeError: 'Series' object has no attribute 'no'
'''

# 2) FEATURE ENGINEERED 'total_rooms' column:

In [0]:
# 2) FEATURE ENGINEERED 'total_rooms' column: Total number of rooms (beds + baths)
df['total_rooms'] = df['bedrooms'] + df['bathrooms']
df.head()

In [68]:
#Check the columns
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space', 'year',
       'has_description', 'total_rooms', 'year_created', 'month_created'],
      dtype='object')

In [0]:
# drop target and categorical columns WHY NOT WORKING!@?#
'''
df.drop(columns=['price','description',
        'display_address',
        'street_address',
        'interest_level']], axis=1)

df.columns
'''

# Fit a linear regression model with at least two features.

## 5 Step Process:

In [0]:
# 5 Step Process:
# 1) Import appropriate estimator class from sklearn

from sklearn.linear_model import LinearRegression

In [0]:
# 2) Instantiate this class
model = LinearRegression()

# 3 WAYS TO SPLIT TRAIN/TEST DATASETS/dataframes:

In [0]:
#3a) Arrange X FEATURES matrices- ONE WAY:

df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True) # change dtype from string to datetime to access month, day, yr etc

# filter out year 2016
df['year_created'] = df['created'].dt.year    # access yr
df = df[df['year_created'] == 2016]

df['month_created'] = df['created'].dt.month  # access month

# filter out months Apr & May for TRAIN set
train = df[(df['month_created'] == 4) | (df['month_created'] == 5)]

# filter out month June for TEST set
test = df[df['month_created'] == 6]

In [0]:
train.shape, test.shape

In [0]:
#3b) Arrange X FEATURES MATRICES- a 2nd WAY

features = ['total_rooms', 'has_description'] # a python list, called features, containing these 2 columns' data

X_train = train[features] # split 2 chosen features' columns' row fields into train dataframe matrix
X_test  =  test[features] # split " " " " " " into test dataframe matrix

X_train, X_test #BEST WAY TO VISUALIZE? shape below? head()?

# Arrange y target Matrix
target = 'price'  # target is an sklearn df? attribute that allows one to access the target values (fields?) aka labels- CHRIS TL said NOT IN GENERAL
                  # else Colab/python (which) recognizes it as one of the columns (color change), and can ALWAYS? set a col to a var w/o calling?/using the df[]?

#Drop target column from features to prevent data leakage and over-fitting
'''
features = df.columns.drop('price',
        'description',
        'display_address',
        'street_address',
        'interest_level')
''' # TypeError: drop() takes from 2 to 3 positional arguments but 6 were given


y_train = train[target]  # predicted y target column (no x, only y) values output for passed in features datapoints in/from TRAIN DATASET/dataframe
y_test  =  test[target]  # predicted y target column (no x, only y) values output for passed in features datapoints in/from TEST DATASET/dataframe


In [0]:
print(f'Linear Regression, dependent upon: {features}')
print('X_train.shape: ', X_train.shape, '\n' ' X_test.shape: ', X_test.shape)

In [0]:
'''
# 3b) a 3rd way to split test/train using the TRAIN_TEST_SPLIT METHOD
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2)
'''

In [0]:
print(df['has_description'])

In [55]:
df['has_description'].dtype

dtype('int64')

In [0]:
df.dtypes

In [0]:
''' DROP obj datatype cols for model
df.drop('description',
        'display_address',
        'street_address',
        'interest_level',
       )

df.columns()
'''

In [100]:
#4) CONVERT categorical columns to numerical for model: One-hot encoding of 'has_description'
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

#y_train_encoded = encoder.fit_transform(y_train)  #KeyError: 'has_description'

print(X_train_encoded.shape, X_test_encoded.shape)

KeyError: ignored

In [0]:
#One-hot encoding: specify 'has_description' column to encode and replace original with encoded version
df['has_description'] = encoder.fit_transform(df['has_description']) #AttributeError: module 'category_encoders' has no attribute 'fit_transform'
df['has_description']  # One hot encoding TRANSFORMED "Yes" into 1 and "No" into 0

In [0]:
#model.fit still gives ValueError: could not convert string to float: 'Yes'- change to 1 = Yes, 0 = No, or One Hot Encode
#trying type casting int64 to float with .astype()



In [98]:
#4a) fit model on TRAIN DATASET
model.fit(X_train_encoded, y_train) # ValueError: could not convert string to float: 'Yes'- change to 1 = Yes, 0 = No, or One Hot Encode

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [99]:
# 5a) calculate Train error
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_train) # get prediction of y target values 
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} percentage points')

ValueError: ignored

In [0]:
# 5b) calculate Test error
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae:.2f} percentage points')

# Get the model's coefficients and intercept.

In [0]:
 model.coef_, model.intercept_

In [0]:
# equation for the model
beta0 = model.intercept_
beta1, beta2 = model.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')


In [0]:
# model's coefficients and intercept. 
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

## Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.

In [0]:
# Regression metrics for TRAIN data
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_train)

mse = mean_squared_error(y_train, y_pred)  #Mean squared error

rmse = np.sqrt(mse)                        #root mean squared Error

mae = mean_absolute_error(y_train, y_pred) #mean absolute Error

r2 = r2_score(y_train, y_pred)             #r^2

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

In [0]:
# Regression metrics for TEST data
y_pred = model.predict(X_test) 

mse = mean_squared_error(y_test, y_pred)  #Mean squared error

rmse = np.sqrt(mse)                       #root mean squared Error

mae = mean_absolute_error(y_test, y_pred) #mean absolute Error

r2 = r2_score(y_test, y_pred)             #r^2

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)